In [ ]:
"""
This code is our finetuning experiment with IDF alignment & finetuning just the embedding.
"""

: 

In [ ]:
SEED = 6

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
root_dir = '/content/drive/My Drive/crossencoder/'
os.chdir(root_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers -q
!pip install pytrec_eval  # To evaluate IR models

  Preparing metadata (setup.py) ... done
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308214 sha256=e7fb7be117ac388ab50be7e906c3f781b882957f9314b28fce86995160f786c0
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec_eval


In [ ]:
import random
import torch
import torch.nn.functional as F
import numpy as np

In [ ]:
# It is important to fix the so we can reproduce our results and debug without random changes in the result.
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
!nvidia-smi

Wed Dec 18 16:18:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0              46W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

# Set up BEIR for evaluation

In [ ]:
!pip install beir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.5 MB/s eta 0:00:00
  Created wheel for beir: filename=beir-2.0.0-py3-none-any.whl size=63550 sha256=f48b734dd43debc1ec270e6f3ddf00388bec953c39800f3dde1ff567784b6128
  Stored in directory: /root/.cache/pip/wheels/1c/14/96/c606ede3c10e9300ef771a6183af09d389459195ff5f854862
Successfully built beir
  Attempting uninstall: fsspec
    Fou

In [ ]:
from beir.datasets.data_loader import GenericDataLoader

#data_path = "datasets/scifact"
DATASET ='nfcorpus'
data_path = f"datasets/{DATASET}"
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test") # or split = "train" or "dev"

/usr/local/lib/python3.10/dist-packages/beir/datasets/data_loader.py:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/3633 [00:00<?, ?it/s]

In [ ]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512



elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


In [ ]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch


In [ ]:

import time

# Sleep for few seconds to let the instance start.
time.sleep(20)

In [ ]:
%%bash

ps -ef | grep elasticsearch

root        4450    4448  0 16:19 ?        00:00:00 grep elasticsearch


In [ ]:
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval
try:


  #### Provide parameters for elastic-search
  hostname = "localhost"
  index_name = f"{DATASET}"
  initialize = True # True, will delete existing index with same name and reindex all documents

  model = BM25(index_name=index_name, hostname=hostname, initialize=initialize)
  retriever = EvaluateRetrieval(model)

  #### Retrieve dense results (format of results is identical to qrels)
  results = retriever.retrieve(corpus, queries)
except Exception as e:
  print(e)


ERROR:root:Unable to create Index in Elastic Search. Reason: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7cee7559cee0>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7cee7559cee0>: Failed to establish a new connection: [Errno 111] Connection refused)
ERROR:root:Unable to create Index in Elastic Search. Reason: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7cee7559cc40>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7cee7559cc40>: Failed to establish a new connection: [Errno 111] Connection refused)
  0%|          | 0/3633 [00:00<?, ?docs/s]

ConnectionError(<urllib3.connection.HTTPConnection object at 0x7cee7559d0c0>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7cee7559d0c0>: Failed to establish a new connection: [Errno 111] Connection refused)


In [ ]:
# Define the file path to load the results from CSV
from beir.retrieval.search.lexical import BM25Search as BM25
from beir.retrieval.evaluation import EvaluateRetrieval
import csv

input_file = f"bm25_results_{DATASET}.csv"

# Initialize an empty dictionary for results
results = {}

# Load results from CSV
with open(input_file, mode="r") as file:
    reader = csv.reader(file)
    next(reader)  # Skip the header row if it exists
    for row in reader:
        query_id = row[0]
        doc_id = row[1]
        score = float(row[2])

        # Check if the query_id is already in the results dictionary
        if query_id not in results:
            results[query_id] = {}

        # Add document ID and its score to the query's result
        results[query_id][doc_id] = score
model = BM25(index_name=index_name, hostname=hostname, initialize=initialize)
retriever = EvaluateRetrieval(model)
#### Evaluate your retrieval using NDCG@k, MAP@K ...
ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)
print(ndcg, _map, recall, precision)

  0%|          | 0/3633 [00:00<?, ?docs/s]
ERROR:root:Unable to create Index in Elastic Search. Reason: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7cee87807310>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7cee87807310>: Failed to establish a new connection: [Errno 111] Connection refused)
ERROR:root:Unable to create Index in Elastic Search. Reason: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7cee756e5360>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7cee756e5360>: Failed to establish a new connection: [Errno 111] Connection refused)


{'NDCG@1': 0.44968, 'NDCG@3': 0.40253, 'NDCG@5': 0.37705, 'NDCG@10': 0.34281, 'NDCG@100': 0.28939, 'NDCG@1000': 0.32065} {'MAP@1': 0.05936, 'MAP@3': 0.10053, 'MAP@5': 0.11329, 'MAP@10': 0.12969, 'MAP@100': 0.15417, 'MAP@1000': 0.16002} {'Recall@1': 0.05936, 'Recall@3': 0.11319, 'Recall@5': 0.13313, 'Recall@10': 0.16603, 'Recall@100': 0.26023, 'Recall@1000': 0.38996} {'P@1': 0.46753, 'P@3': 0.37771, 'P@5': 0.32273, 'P@10': 0.24708, 'P@100': 0.06828, 'P@1000': 0.01177}


In [ ]:
#align+finetune embedding
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, TrainerCallback, TrainerState, TrainerControl
from beir.reranking.models import CrossEncoder
from beir.reranking import Rerank


def evaluate_beir(fine_tuned_model_path,NUM_NEG_SAMPLES):
    cross_encoder_model = CrossEncoder(fine_tuned_model_path)
    reranker = Rerank(cross_encoder_model, batch_size=128)
    # Rerank top-100 results using the reranker provided
    rerank_results = reranker.rerank(corpus, queries, results, top_k=20)
    #### Evaluate your retrieval using NDCG@k, MAP@K ...
    ndcg, _map, recall, precision = EvaluateRetrieval.evaluate(qrels, rerank_results, retriever.k_values)
    print(ndcg)
    print(_map)
    print(recall)
    print(precision)
    return ndcg['NDCG@1']
#best NDCG@1 was 0.653333

# **SCIFACT**


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "cross-encoder/ms-marco-MiniLM-L-12-v2"#''microsoft/MiniLM-L12-H384-uncased' #"cross-encoder/ms-marco-MiniLM-L-12-v2"#'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
DATASET  = 'nfcorpus'

In [ ]:
import pandas as pd
import random


def make_negative_samples(qrels,output_file_path):
    # Combine test and train qrels if needed
    # qrels = pd.concat([qrels_test, qrels_train])

    # Get all unique corpus IDs for negative sampling
    all_corpus_ids = set(qrels['corpus-id'].unique())

    # Initialize a list to store the rows for the new .tsv file
    rows = []
    random.seed(36)

    # Loop over each unique query-id
    for query_id, group in qrels.groupby('query-id'):
        # Get the list of positive corpus IDs for this query-id
        positive_ids = group['corpus-id'].tolist()
        random.seed(36)
        # For each positive corpus-id, sample 3 negative corpus-ids
        for pos_id in positive_ids:

            # Sample 3 negative corpus-ids (not in the positive set for this query)
            negative_ids = random.sample(list(all_corpus_ids - set(positive_ids)), NUM_NEG_SAMPLES)

            # Add a new row with query-id, positive-corpus-id, and each negative-corpus-id
            for neg_id in negative_ids:
                rows.append([query_id, pos_id, neg_id])

    # Create a DataFrame and save it as a .tsv file
    output_df = pd.DataFrame(rows, columns=['query-id', 'positive-corpus-id', 'negative-corpus-id'])
    output_df.to_csv(output_file_path, sep='\t', index=False)


import json
import random



def serialize_query_passage(query, passage):
    return f'Query: {query} Passage: {passage}'

def load_corpus(path):

    """Load the corpus from a JSONL file and return a dictionary mapping corpus-id to passage text."""
    corpus = {}
    with open(path) as f:
        for line in f:
            entry = json.loads(line)
            corpus[entry["_id"]] = entry['text']  # Adjust 'text' to the actual field name in your JSONL file
    return corpus

def load_queries(path):

    """Load the queries from a JSONL file and return a dictionary mapping qid to query text."""
    queries = {}
    with open(path) as f:
        for line in f:
            entry = json.loads(line)
            queries[entry["_id"]] = entry['text']  # Adjust 'text' to the actual field name in your JSONL file
    return queries

def load_examples(path, corpus_path, queries_path, num_neg_samples):
    random.seed(36)
    """Load examples from a TSV file using the loaded queries and corpus data."""
    # Load corpus and queries data
    corpus = load_corpus(corpus_path)
    queries = load_queries(queries_path)

    x = []
    y = []
    with open(path) as f:
        next(f)  # Skip the first line
        for line in f:
            qid, pos_id, *_ = line.strip().split('\t')

            # Retrieve the query and positive passage using the IDs
            try:
              query = queries[qid]
              positive_passage = corpus[pos_id]

              # Append the positive example
              x.append(serialize_query_passage(query, positive_passage))
              y.append(True)

              # Sample multiple negative corpus IDs
              possible_neg_ids = list(corpus.keys() - {pos_id})
              negative_ids = random.sample(possible_neg_ids, num_neg_samples)

              # Append each negative example
              for neg_id in negative_ids:
                  negative_passage = corpus[neg_id]
                  x.append(serialize_query_passage(query, negative_passage))
                  y.append(False)
            except KeyError as e:
                pass

    return x, y




In [ ]:
# Set a global seed
"""
SEED = 36
random.seed(SEED)
np.random.seed(SEED)

max_train = 100_000
max_valid = 10_000

NUM_NEG_SAMPLES = 8
# Load the qrels data
qrels_test = pd.read_csv('datasets/scifact/qrels/test.tsv', sep='\t', names=['query-id', 'corpus-id', 'score'])
qrels_train = pd.read_csv('datasets/scifact/qrels/train.tsv', sep='\t', names=['query-id', 'corpus-id', 'score'])

train_fp = f'datasets/{DATASET}/query_pos_neg_samples_{NUM_NEG_SAMPLES}_train.tsv'
test_fp  = f'datasets/{DATASET}/query_pos_neg_samples_{NUM_NEG_SAMPLES}_test.tsv'
make_negative_samples(qrels_train,train_fp)
make_negative_samples(qrels_test,test_fp)
# Load the examples
x_train, y_train = load_examples(
    train_fp,
    'datasets/scifact/corpus.jsonl',
    'datasets/scifact/queries.jsonl',
    NUM_NEG_SAMPLES
)

# Assuming you have a similar validation set, load it here (otherwise, initialize empty lists)
x_valid, y_valid = load_examples(
    test_fp,
    f'datasets/{DATASET}/corpus.jsonl',
    f'datasets/{DATASET}/queries.jsonl',
    NUM_NEG_SAMPLES
)

x_valid = x_train[-max_valid:]
y_valid = y_train[-max_valid:]
x_train = x_train[:max_train]
y_train = y_train[:max_train]
"""
import pandas as pd
NUM_NEG_SAMPLES = 3
# Define file paths
train_csv_path = f'finetune_{DATASET}/{NUM_NEG_SAMPLES}_train_examples.csv'
valid_csv_path = f'finetune_{DATASET}/{NUM_NEG_SAMPLES}_valid_examples.csv'

# Load training data
train_df = pd.read_csv(train_csv_path)
x_train = train_df['text'].tolist()
y_train = train_df['label'].tolist()
print(f"Training data loaded from {train_csv_path}")

# Load validation data
valid_df = pd.read_csv(valid_csv_path)
x_valid = valid_df['text'].tolist()
y_valid = valid_df['label'].tolist()
print(f"Validation data loaded from {valid_csv_path}")

# Verify loaded data
print(f"Number of training examples: {len(x_train)}")
print(f"Number of validation examples: {len(x_valid)}")

# Print stats and example outputs
print(len(x_train), 'training examples.')
print(len(x_valid), 'valid examples.')

print('\nFirst 3 training examples:')
for x, y in zip(x_train[:20], y_train[:20]):
    print(f'Label: {y} Input: "{x}"')

print('\n' + 100 * '-' + '\n\nFirst 3 validation examples:')
for x, y in zip(x_valid[:20], y_valid[:20]):
    print(f'Label: {y} Input: "{x}"')

Training data loaded from finetune_nfcorpus/3_train_examples.csv
Validation data loaded from finetune_nfcorpus/3_valid_examples.csv
Number of training examples: 100000
Number of validation examples: 10000
100000 training examples.
10000 valid examples.

First 3 training examples:
Label: True Input: "Query: How Contaminated Are Our Children? Passage: Background In the absence of current cumulative dietary exposure assessments, this analysis was conducted to estimate exposure to multiple dietary contaminants for children, who are more vulnerable to toxic exposure than adults. Methods We estimated exposure to multiple food contaminants based on dietary data from preschool-age children (2–4 years, n=207), school-age children (5–7 years, n=157), parents of young children (n=446), and older adults (n=149). We compared exposure estimates for eleven toxic compounds (acrylamide, arsenic, lead, mercury, chlorpyrifos, permethrin, endosulfan, dieldrin, chlordane, DDE, and dioxin) based on self-rep

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "cross-encoder/ms-marco-MiniLM-L-12-v2"#''microsoft/MiniLM-L12-H384-uncased' #"cross-encoder/ms-marco-MiniLM-L-12-v2"#'
tokenizer = AutoTokenizer.from_pretrained(model_name)

from statistics import mean, stdev

lengths = [len(tokens) for tokens in tokenizer(x_train[:1_000])['input_ids']]
print(f'Mean length in tokens: {mean(lengths):0.2f}')
print(f'Stdev length in tokens: {stdev(lengths):0.2f}')

Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors


Mean length in tokens: 349.35
Stdev length in tokens: 142.61


In [ ]:
# Since the mean passage length is ~380 tokens, a maximum of 460 tokens should be more than enough to acommodate
# query+passage tokens in the input.
max_length = 430
x_train_tokenized = tokenizer(x_train, max_length=max_length, truncation=True)
x_valid_tokenized = tokenizer(x_valid, max_length=max_length, truncation=True)

In [ ]:
# Since the mean passage length is ~380 tokens, a maximum of 460 tokens should be more than enough to acommodate
# query+passage tokens in the input.
max_length = 430
x_train_tokenized = tokenizer(x_train, max_length=max_length, truncation=True)
x_valid_tokenized = tokenizer(x_valid, max_length=max_length, truncation=True)

In [ ]:



from torch.utils import data

class Dataset(data.Dataset):
    def __init__(self, examples, targets=None):
        self.examples = examples
        self.targets = targets

    def __len__(self):
        return len(self.examples['input_ids'])

    def __getitem__(self, idx):
        label = 0.0
        if self.targets is not None:
            label = float(self.targets[idx])

        return {
            'input_ids': self.examples['input_ids'][idx],
            'attention_mask': self.examples['attention_mask'][idx],
            'labels': float(label),
        }

dataset_train = Dataset(x_train_tokenized, y_train)
assert len(dataset_train[0]['input_ids']) > 0
assert len(dataset_train[1]['attention_mask']) > 0
assert type(dataset_train[2]['labels']) == float

dataset_train = Dataset(x_train_tokenized, y_train)
dataset_valid = Dataset(x_valid_tokenized, y_valid)

from transformers import BatchEncoding

def collate_fn(batch):
    return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))



generator = torch.Generator()
generator.manual_seed(SEED)

dataloader_train = data.DataLoader(dataset_train, batch_size=32, shuffle=True, collate_fn=collate_fn,generator=generator)
dataloader_valid = data.DataLoader(dataset_valid, batch_size=32, shuffle=False, collate_fn=collate_fn,generator=generator)

for batch in dataloader_train:
    assert batch['input_ids'].shape[0] <= dataloader_train.batch_size
    assert batch['input_ids'].shape[1] <= max_length
    break


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:

def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for batch in tqdm(dataloader, mininterval=0.5, desc=set_name, disable=False):
            # Move batch to device
            batch = {k: v.to(device) for k, v in batch.items()}

            # Forward pass
            outputs = model(**batch)

            # Apply sigmoid to logits for binary relevance scores
            predictions = torch.sigmoid(outputs.logits).squeeze()

            # Calculate MSE loss with sigmoid-transformed predictions
            loss_val = F.mse_loss(predictions, batch['labels'].float())
            losses.append(loss_val.cpu().item())

            # Convert probabilities to binary predictions (threshold at 0.5)
            preds = (predictions > 0.5).long()

            # Calculate accuracy
            correct += (preds == batch['labels']).sum().item()

    # Print average loss and accuracy
    avg_loss = mean(losses)
    accuracy = correct / len(dataloader.dataset)
    print(f'{set_name} loss: {avg_loss:.3f}; {set_name} accuracy: {accuracy:.3f}')
    return avg_loss, accuracy


In [ ]:
import torch
from torch import nn, optim
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from statistics import mean
import os
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Set the seed for reproducibility

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # if using CUDA

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

print('Parameters:', model.num_parameters())

# Training configuration
epochs = 30
learning_rate = 2e-5  # Adjust as necessary
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
num_training_steps = epochs * len(dataloader_train)
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# Track the best NDCG score
best_ndcg = float("-inf")
fine_tuned_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/temp_model_epoch_NDCG_full_fixedrandom"
best_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/best_model_full_weights_NDCG_fixedrandom"
os.makedirs(fine_tuned_model_path, exist_ok=True)
os.makedirs(best_model_path, exist_ok=True)

# Initial validation to verify evaluation code
evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

# Training loop
for epoch in range(epochs):
    model.train()
    train_losses = []
    print(f"Epoch {epoch + 1}/{epochs}")

    for batch in tqdm(dataloader_train, desc="Train"):
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(**batch)

        # Apply sigmoid to logits for binary relevance scores
        predictions = torch.sigmoid(outputs.logits).squeeze()

        # Calculate MSE loss with sigmoid-transformed predictions
        loss = nn.functional.mse_loss(predictions, batch['labels'].float())
        train_losses.append(loss.cpu().item())

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Print average training loss for the epoch
    avg_train_loss = mean(train_losses)
    print(f"Epoch: {epoch + 1} Training loss: {avg_train_loss:.2f}")

    # Save the model at the end of each epoch
    model.save_pretrained(fine_tuned_model_path)
    tokenizer.save_pretrained(fine_tuned_model_path)

    # Evaluate NDCG using the saved model path
    ndcg = evaluate_beir(fine_tuned_model_path, NUM_NEG_SAMPLES=NUM_NEG_SAMPLES)
    print(f"NDCG@1 for epoch {epoch + 1}: {ndcg:.4f}")
    evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')


    # Check if this NDCG score is the highest seen so far
    if ndcg > best_ndcg:
        best_ndcg = ndcg
        # Save the model with the highest NDCG
        model.save_pretrained(best_model_path)
        tokenizer.save_pretrained(best_model_path)
        print(f"New best model saved with NDCG@1: {ndcg:.4f}")

print("Training complete. Best model saved at:", best_model_path)


model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Parameters: 33360385


Valid:   0%|          | 0/313 [00:00<?, ?it/s]

Valid loss: 0.247; Valid accuracy: 0.716
Epoch 1/30


Train:   0%|          | 0/3125 [00:00<?, ?it/s]

...

Epoch: 30 Training loss: 0.04


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

{'NDCG@1': 0.48214, 'NDCG@3': 0.43083, 'NDCG@5': 0.40088, 'NDCG@10': 0.35957, 'NDCG@100': 0.25038, 'NDCG@1000': 0.24337}
{'MAP@1': 0.06354, 'MAP@3': 0.1051, 'MAP@5': 0.11989, 'MAP@10': 0.13608, 'MAP@100': 0.14589, 'MAP@1000': 0.14589}
{'Recall@1': 0.06354, 'Recall@3': 0.11618, 'Recall@5': 0.14106, 'Recall@10': 0.16893, 'Recall@100': 0.19034, 'Recall@1000': 0.19034}
{'P@1': 0.5, 'P@3': 0.40043, 'P@5': 0.34156, 'P@10': 0.25844, 'P@100': 0.03481, 'P@1000': 0.00348}
NDCG@1 for epoch 30: 0.4821


Valid:   0%|          | 0/313 [00:00<?, ?it/s]

Valid loss: 0.226; Valid accuracy: 0.746
Training complete. Best model saved at: finetune_nfcorpus/neg_samples_3_seed6/best_model_full_weights_NDCG_fixedrandom


In [ ]:
import os
import torch
from torch import nn, optim
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from statistics import mean
import pandas as pd  # Make sure to import pandas
seed = 36
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if using CUDA
# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

# Initialize Embedding Weights with SVD-based IDF Modification
# Step 1: Extract the original embedding matrix W_E
W_E = model.get_input_embeddings().weight.detach().cpu()  # Original embedding matrix W_E

# Step 2: Perform SVD on the embedding matrix W_E
U, S, Vh = torch.linalg.svd(W_E, full_matrices=False)  # Perform SVD on W_E

# Step 3: Load IDF scores and normalize them
idf_file_path = f"{DATASET}_idf_scores.csv"
idf_scores = pd.read_csv(idf_file_path).iloc[:, 1].tolist()  # Assuming IDF values are in the second column
idf_tensor = -1 * torch.tensor(idf_scores, dtype=torch.float32)

# Create a mask for entries in idf_tensor that are equal to -1.0
mask = idf_tensor == -1.0
# Count the number of masked elements
num_masked_elements = mask.sum().item()
print("Number of masked elements:", num_masked_elements)
print("Total number of elements in idf_tensor:", idf_tensor.shape[0])

# Normalize idf_tensor to match the distribution of the original U[:, 0]
original_mean, original_std = U[:, 0].mean(), U[:, 0].std()
idf_mean, idf_std = idf_tensor.mean(), idf_tensor.std()
normalized_idf_tensor = (idf_tensor - idf_mean) / idf_std * original_std + original_mean

# Use the mask to retain original U[:, 0] values for entries where idf_tensor == -1.0
normalized_idf_tensor[mask] = U[:, 0][mask]

# Set the first column of U to the modified IDF values
U[:, 0] = normalized_idf_tensor

# Step 4: Reconstruct W_E with the modified U
W_E_modified = torch.mm(U, torch.diag(S)) @ Vh  # Reconstruct W_E with modified U
model.get_input_embeddings().weight.data = W_E_modified.to(model.device)  # Assign modified W_E back to model

# Step 5: Save the modified model and tokenizer if desired
modified_model_path = "modified_cross_encoder_model"
os.makedirs(modified_model_path, exist_ok=True)
model.save_pretrained(modified_model_path)

# Then start training

# Freeze all layers except the embeddings
for name, param in model.named_parameters():
    if 'embeddings' not in name:  # Freeze all layers except embeddings
        param.requires_grad = False

# Verify the number of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable Parameters: {trainable_params}')

# Training configuration
epochs = 30
learning_rate = 2e-5  # Adjust as necessary
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
num_training_steps = epochs * len(dataloader_train)
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# Track the best NDCG score
best_ndcg = float("-inf")
fine_tuned_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/temp_model_epoch_NDCG_ALIGNED_full"
best_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/best_model_ALIGNED_full_NDCG"
os.makedirs(fine_tuned_model_path, exist_ok=True)
os.makedirs(best_model_path, exist_ok=True)

# Initial validation to verify evaluation code
evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

# Training loop
for epoch in range(epochs):
    model.train()
    train_losses = []
    print(f"Epoch {epoch + 1}/{epochs}")

    for batch in tqdm(dataloader_train, desc="Train"):
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(**batch)

        # Apply sigmoid to logits for binary relevance scores
        predictions = torch.sigmoid(outputs.logits).squeeze()

        # Calculate MSE loss with sigmoid-transformed predictions
        loss = nn.functional.mse_loss(predictions, batch['labels'].float())
        train_losses.append(loss.cpu().item())

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Print average training loss for the epoch
    avg_train_loss = mean(train_losses)
    print(f"Epoch: {epoch + 1} Training loss: {avg_train_loss:.2f}")

    # Save the model at the end of each epoch
    model.save_pretrained(fine_tuned_model_path)
    tokenizer.save_pretrained(fine_tuned_model_path)

    # Evaluate NDCG using the saved model path
    ndcg = evaluate_beir(fine_tuned_model_path, NUM_NEG_SAMPLES=NUM_NEG_SAMPLES)
    print(f"NDCG@1 for epoch {epoch + 1}: {ndcg:.4f}")
    evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')


    # Check if this NDCG score is the highest seen so far
    if ndcg > best_ndcg:
        best_ndcg = ndcg
        # Save the model with the highest NDCG
        model.save_pretrained(best_model_path)
        tokenizer.save_pretrained(best_model_path)
        print(f"New best model saved with NDCG@1: {ndcg:.4f}")

print("Training complete. Best model saved at:", best_model_path)

Number of masked elements: 19436
Total number of elements in idf_tensor: 30522
Trainable Parameters: 11918592


Valid:   0%|          | 0/313 [00:00<?, ?it/s]

Valid loss: 0.243; Valid accuracy: 0.731
Epoch 1/30


Train:   0%|          | 0/3125 [00:00<?, ?it/s]

Epoch: 1 Training loss: 0.24


...

Epoch: 30 Training loss: 0.09


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

{'NDCG@1': 0.49351, 'NDCG@3': 0.43713, 'NDCG@5': 0.40082, 'NDCG@10': 0.35579, 'NDCG@100': 0.25225, 'NDCG@1000': 0.24532}
{'MAP@1': 0.06306, 'MAP@3': 0.10624, 'MAP@5': 0.11972, 'MAP@10': 0.13481, 'MAP@100': 0.14566, 'MAP@1000': 0.14566}
{'Recall@1': 0.06306, 'Recall@3': 0.11604, 'Recall@5': 0.13922, 'Recall@10': 0.16388, 'Recall@100': 0.19034, 'Recall@1000': 0.19034}
{'P@1': 0.51299, 'P@3': 0.40476, 'P@5': 0.33571, 'P@10': 0.24903, 'P@100': 0.03481, 'P@1000': 0.00348}
NDCG@1 for epoch 30: 0.4935


Valid:   0%|          | 0/313 [00:00<?, ?it/s]

Valid loss: 0.198; Valid accuracy: 0.724
Training complete. Best model saved at: finetune_nfcorpus/neg_samples_3_seed6/best_model_ALIGNED_full_NDCG


In [ ]:

import torch
from torch import nn, optim
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from statistics import mean
import os
# Set the seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)  # if using CUDA

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
# Freeze all layers except the embeddings
for name, param in model.named_parameters():
    if 'embeddings' not in name:  # Freeze all layers except embeddings
        param.requires_grad = False

print('Parameters:', model.num_parameters())

# Training configuration
epochs =30
learning_rate = 2e-5  # Adjust as necessary
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
num_training_steps = epochs * len(dataloader_train)
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# Track the best NDCG score
best_ndcg = float("-inf")
fine_tuned_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/temp_model_epoch_NDCG_embedding_only"
best_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/best_model_embedding_only_NDCG"
os.makedirs(fine_tuned_model_path, exist_ok=True)
os.makedirs(best_model_path, exist_ok=True)

# Initial validation to verify evaluation code
evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

# Training loop
for epoch in range(epochs):
    model.train()
    train_losses = []
    print(f"Epoch {epoch + 1}/{epochs}")

    for batch in tqdm(dataloader_train, desc="Train"):
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(**batch)

        # Apply sigmoid to logits for binary relevance scores
        predictions = torch.sigmoid(outputs.logits).squeeze()

        # Calculate MSE loss with sigmoid-transformed predictions
        loss = nn.functional.mse_loss(predictions, batch['labels'].float())
        train_losses.append(loss.cpu().item())

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Print average training loss for the epoch
    avg_train_loss = mean(train_losses)
    print(f"Epoch: {epoch + 1} Training loss: {avg_train_loss:.2f}")

    # Save the model at the end of each epoch
    model.save_pretrained(fine_tuned_model_path)
    tokenizer.save_pretrained(fine_tuned_model_path)

    # Evaluate NDCG using the saved model path
    ndcg = evaluate_beir(fine_tuned_model_path, NUM_NEG_SAMPLES=NUM_NEG_SAMPLES)
    print(f"NDCG@1 for epoch {epoch + 1}: {ndcg:.4f}")

    # Check if this NDCG score is the highest seen so far
    if ndcg > best_ndcg:
        best_ndcg = ndcg
        # Save the model with the highest NDCG
        model.save_pretrained(best_model_path)
        tokenizer.save_pretrained(best_model_path)
        print(f"New best model saved with NDCG@1: {ndcg:.4f}")

print("Training complete. Best model saved at:", best_model_path)


Parameters: 33360385


Valid:   0%|          | 0/313 [00:00<?, ?it/s]

Valid loss: 0.247; Valid accuracy: 0.716
Epoch 1/30


Train:   0%|          | 0/3125 [00:00<?, ?it/s]

Epoch: 1 Training loss: 0.24


...

{'NDCG@1': 0.47727, 'NDCG@3': 0.42438, 'NDCG@5': 0.39538, 'NDCG@10': 0.35293, 'NDCG@100': 0.24972, 'NDCG@1000': 0.2429}
{'MAP@1': 0.05926, 'MAP@3': 0.10382, 'MAP@5': 0.11818, 'MAP@10': 0.13426, 'MAP@100': 0.1443, 'MAP@1000': 0.1443}
{'Recall@1': 0.05926, 'Recall@3': 0.11428, 'Recall@5': 0.13902, 'Recall@10': 0.16633, 'Recall@100': 0.19034, 'Recall@1000': 0.19034}
{'P@1': 0.49351, 'P@3': 0.39502, 'P@5': 0.33636, 'P@10': 0.25162, 'P@100': 0.03481, 'P@1000': 0.00348}
NDCG@1 for epoch 29: 0.4773
Epoch 30/30


Train:   0%|          | 0/3125 [00:00<?, ?it/s]

Epoch: 30 Training loss: 0.09


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

{'NDCG@1': 0.47727, 'NDCG@3': 0.42438, 'NDCG@5': 0.39538, 'NDCG@10': 0.35293, 'NDCG@100': 0.24972, 'NDCG@1000': 0.2429}
{'MAP@1': 0.05926, 'MAP@3': 0.10382, 'MAP@5': 0.11818, 'MAP@10': 0.13426, 'MAP@100': 0.1443, 'MAP@1000': 0.1443}
{'Recall@1': 0.05926, 'Recall@3': 0.11428, 'Recall@5': 0.13902, 'Recall@10': 0.16633, 'Recall@100': 0.19034, 'Recall@1000': 0.19034}
{'P@1': 0.49351, 'P@3': 0.39502, 'P@5': 0.33636, 'P@10': 0.25162, 'P@100': 0.03481, 'P@1000': 0.00348}
NDCG@1 for epoch 30: 0.4773
Training complete. Best model saved at: finetune_nfcorpus/neg_samples_3_seed6/best_model_embedding_only_NDCG


# IDF-ALIGNMENT + FINETUNING SCIFACT

In [ ]:
import os
import torch
from torch import nn, optim
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from statistics import mean
import pandas as pd  # Make sure to import pandas
seed = 36
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if using CUDA
# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

# Initialize Embedding Weights with SVD-based IDF Modification
# Step 1: Extract the original embedding matrix W_E
W_E = model.get_input_embeddings().weight.detach().cpu()  # Original embedding matrix W_E

# Step 2: Perform SVD on the embedding matrix W_E
U, S, Vh = torch.linalg.svd(W_E, full_matrices=False)  # Perform SVD on W_E

# Step 3: Load IDF scores and normalize them
idf_file_path = f"{DATASET}_idf_scores.csv"
idf_scores = pd.read_csv(idf_file_path).iloc[:, 1].tolist()  # Assuming IDF values are in the second column
idf_tensor = -1 * torch.tensor(idf_scores, dtype=torch.float32)

# Create a mask for entries in idf_tensor that are equal to -1.0
mask = idf_tensor == -1.0
# Count the number of masked elements
num_masked_elements = mask.sum().item()
print("Number of masked elements:", num_masked_elements)
print("Total number of elements in idf_tensor:", idf_tensor.shape[0])

# Normalize idf_tensor to match the distribution of the original U[:, 0]
original_mean, original_std = U[:, 0].mean(), U[:, 0].std()
idf_mean, idf_std = idf_tensor.mean(), idf_tensor.std()
normalized_idf_tensor = (idf_tensor - idf_mean) / idf_std * original_std + original_mean

# Use the mask to retain original U[:, 0] values for entries where idf_tensor == -1.0
normalized_idf_tensor[mask] = U[:, 0][mask]

# Set the first column of U to the modified IDF values
U[:, 0] = normalized_idf_tensor

# Step 4: Reconstruct W_E with the modified U
W_E_modified = torch.mm(U, torch.diag(S)) @ Vh  # Reconstruct W_E with modified U
model.get_input_embeddings().weight.data = W_E_modified.to(model.device)  # Assign modified W_E back to model

# Step 5: Save the modified model and tokenizer if desired
modified_model_path = "modified_cross_encoder_model"
os.makedirs(modified_model_path, exist_ok=True)
model.save_pretrained(modified_model_path)

# Then start training

# Freeze all layers except the embeddings
for name, param in model.named_parameters():
    if 'embeddings' not in name:  # Freeze all layers except embeddings
        param.requires_grad = False

# Verify the number of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable Parameters: {trainable_params}')

# Training configuration
epochs =30
learning_rate = 2e-5  # Adjust as necessary
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
num_training_steps = epochs * len(dataloader_train)
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# Track the best NDCG score
best_ndcg = float("-inf")
fine_tuned_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/temp_model_epoch_NDCG_ALIGNED_embedding_only"
best_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/best_model_ALIGNED_embedding_only_NDCG"
os.makedirs(fine_tuned_model_path, exist_ok=True)
os.makedirs(best_model_path, exist_ok=True)

# Initial validation to verify evaluation code
evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

# Training loop
for epoch in range(epochs):
    model.train()
    train_losses = []
    print(f"Epoch {epoch + 1}/{epochs}")

    for batch in tqdm(dataloader_train, desc="Train"):
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(**batch)

        # Apply sigmoid to logits for binary relevance scores
        predictions = torch.sigmoid(outputs.logits).squeeze()

        # Calculate MSE loss with sigmoid-transformed predictions
        loss = nn.functional.mse_loss(predictions, batch['labels'].float())
        train_losses.append(loss.cpu().item())

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()
    # Print average training loss for the epoch
    avg_train_loss = mean(train_losses)
    print(f"Epoch: {epoch + 1} Training loss: {avg_train_loss:.2f}")

    # Save the model at the end of each epoch
    model.save_pretrained(fine_tuned_model_path)
    tokenizer.save_pretrained(fine_tuned_model_path)

    # Evaluate NDCG using the saved model path
    ndcg = evaluate_beir(fine_tuned_model_path, NUM_NEG_SAMPLES=NUM_NEG_SAMPLES)
    print(f"NDCG@1 for epoch {epoch + 1}: {ndcg:.4f}")
    evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')


    # Check if this NDCG score is the highest seen so far
    if ndcg > best_ndcg:
        best_ndcg = ndcg
        # Save the model with the highest NDCG
        model.save_pretrained(best_model_path)
        tokenizer.save_pretrained(best_model_path)
        print(f"New best model saved with NDCG@1: {ndcg:.4f}")

print("Training complete. Best model saved at:", best_model_path)


Number of masked elements: 19436
Total number of elements in idf_tensor: 30522
Trainable Parameters: 11918592


Valid:   0%|          | 0/313 [00:00<?, ?it/s]

Valid loss: 0.243; Valid accuracy: 0.731
Epoch 1/30


Train:   0%|          | 0/3125 [00:00<?, ?it/s]

Epoch: 1 Training loss: 0.24


...

Epoch: 30 Training loss: 0.09


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

{'NDCG@1': 0.48214, 'NDCG@3': 0.42705, 'NDCG@5': 0.39668, 'NDCG@10': 0.35488, 'NDCG@100': 0.25042, 'NDCG@1000': 0.24344}
{'MAP@1': 0.06166, 'MAP@3': 0.10446, 'MAP@5': 0.1187, 'MAP@10': 0.13445, 'MAP@100': 0.14479, 'MAP@1000': 0.14479}
{'Recall@1': 0.06166, 'Recall@3': 0.11458, 'Recall@5': 0.13896, 'Recall@10': 0.16489, 'Recall@100': 0.19034, 'Recall@1000': 0.19034}
{'P@1': 0.5, 'P@3': 0.3961, 'P@5': 0.33377, 'P@10': 0.25227, 'P@100': 0.03481, 'P@1000': 0.00348}
NDCG@1 for epoch 30: 0.4821


Valid:   0%|          | 0/313 [00:00<?, ?it/s]

Valid loss: 0.200; Valid accuracy: 0.722
Training complete. Best model saved at: finetune_nfcorpus/neg_samples_3_seed6/best_model_ALIGNED_embedding_only_NDCG


In [ ]:
"""
import torch
from torch import nn, optim
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from statistics import mean
import os
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
seed = SEED
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if using CUDA
# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

# Initialize Embedding Weights with SVD-based IDF Modification
# Step 1: Extract the original embedding matrix W_E
W_E = model.get_input_embeddings().weight.detach().cpu()  # Original embedding matrix W_E

# Step 2: Perform SVD on the embedding matrix W_E
U, S, Vh = torch.linalg.svd(W_E, full_matrices=False)  # Perform SVD on W_E

# Step 3: Load IDF scores and normalize them
idf_file_path = f"{DATASET}_idf_scores.csv"
idf_scores = pd.read_csv(idf_file_path).iloc[:, 1].tolist()  # Assuming IDF values are in the second column
idf_tensor = -1 * torch.tensor(idf_scores, dtype=torch.float32)

# Create a mask for entries in idf_tensor that are equal to -1.0
mask = idf_tensor == -1.0
# Count the number of masked elements
num_masked_elements = mask.sum().item()
print("Number of masked elements:", num_masked_elements)
print("Total number of elements in idf_tensor:", idf_tensor.shape[0])

# Normalize idf_tensor to match the distribution of the original U[:, 0]
original_mean, original_std = U[:, 0].mean(), U[:, 0].std()
idf_mean, idf_std = idf_tensor.mean(), idf_tensor.std()
normalized_idf_tensor = (idf_tensor - idf_mean) / idf_std * original_std + original_mean

# Use the mask to retain original U[:, 0] values for entries where idf_tensor == -1.0
normalized_idf_tensor[mask] = U[:, 0][mask]

# Set the first column of U to the modified IDF values
U[:, 0] = normalized_idf_tensor

# Step 4: Reconstruct W_E with the modified U
W_E_modified = torch.mm(U, torch.diag(S)) @ Vh  # Reconstruct W_E with modified U
model.get_input_embeddings().weight.data = W_E_modified.to(model.device)  # Assign modified W_E back to model

# Step 5: Save the modified model and tokenizer if desired
modified_model_path = "modified_cross_encoder_model"
os.makedirs(modified_model_path, exist_ok=True)
model.save_pretrained(modified_model_path)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable Parameters: {trainable_params}')

# Training configuration
epochs = 10
learning_rate = 2e-5  # Adjust as necessary for training embeddings
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)  # Optimizer only for trainable params
num_training_steps = epochs * len(dataloader_train)
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# Track the best validation loss
best_val_loss = float("inf")
best_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/best_model_aligned_full_NDCG_fixedrandom"  # Path to save the best model
# Track the best NDCG score
best_ndcg = float("-inf")
fine_tuned_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/temp_model_epoch_NDCG_aligned_full_fixedrandom"

os.makedirs(best_model_path, exist_ok=True)
# Initial validation
evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

# Training loop
for epoch in range(epochs):
    model.train()
    train_losses = []
    print(f"Epoch {epoch + 1}/{epochs}")

    for batch in tqdm(dataloader_train, desc="Train"):
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(**batch)

        # Apply sigmoid to logits for binary relevance scores
        predictions = torch.sigmoid(outputs.logits).squeeze()

        # Calculate MSE loss with sigmoid-transformed predictions
        loss = nn.functional.mse_loss(predictions, batch['labels'].float())
        train_losses.append(loss.cpu().item())

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Print average training loss for the epoch
    avg_train_loss = mean(train_losses)
    print(f"Epoch: {epoch + 1} Training loss: {avg_train_loss:.2f}")

    # Save the model at the end of each epoch
    model.save_pretrained(fine_tuned_model_path)
    tokenizer.save_pretrained(fine_tuned_model_path)

    # Evaluate NDCG using the saved model path
    ndcg = evaluate_beir(fine_tuned_model_path, NUM_NEG_SAMPLES=NUM_NEG_SAMPLES)
    print(f"NDCG@1 for epoch {epoch + 1}: {ndcg:.4f}")
    evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')


    # Check if this NDCG score is the highest seen so far
    if ndcg > best_ndcg:
        best_ndcg = ndcg
        # Save the model with the highest NDCG
        model.save_pretrained(best_model_path)
        tokenizer.save_pretrained(best_model_path)
        print(f"New best model saved with NDCG@1: {ndcg:.4f}")

print("Training complete. Best model saved at:", best_model_path)
"""

'\nimport torch\nfrom torch import nn, optim\nfrom tqdm.auto import tqdm\nfrom transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup\nfrom statistics import mean\nimport os\ntorch.backends.cudnn.deterministic = True\ntorch.backends.cudnn.benchmark = False\nseed = SEED\nrandom.seed(seed)\nnp.random.seed(seed)\ntorch.manual_seed(seed)\ntorch.cuda.manual_seed_all(seed)  # if using CUDA\n# Set up device\ndevice = torch.device(\'cuda\' if torch.cuda.is_available() else \'cpu\')\n\nmodel = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)\n\n# Initialize Embedding Weights with SVD-based IDF Modification\n# Step 1: Extract the original embedding matrix W_E\nW_E = model.get_input_embeddings().weight.detach().cpu()  # Original embedding matrix W_E\n\n# Step 2: Perform SVD on the embedding matrix W_E\nU, S, Vh = torch.linalg.svd(W_E, full_matrices=False)  # Perform SVD on W_E\n\n# Step 3: Load IDF scores and normalize them\nidf_file_path 

In [ ]:
"""
import torch
from torch import nn, optim
from tqdm.auto import tqdm
from transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup
from statistics import mean
import os
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
seed = SEED
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if using CUDA
# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

# Initialize Embedding Weights with SVD-based IDF Modification
# Step 1: Extract the original embedding matrix W_E
W_E = model.get_input_embeddings().weight.detach().cpu()  # Original embedding matrix W_E

# Step 2: Perform SVD on the embedding matrix W_E
U, S, Vh = torch.linalg.svd(W_E, full_matrices=False)  # Perform SVD on W_E

# Step 3: Load IDF scores and normalize them
idf_file_path = f"{DATASET}_idf_scores.csv"
idf_scores = pd.read_csv(idf_file_path).iloc[:, 1].tolist()  # Assuming IDF values are in the second column
idf_tensor = -1 * torch.tensor(idf_scores, dtype=torch.float32)

# Create a mask for entries in idf_tensor that are equal to -1.0
mask = idf_tensor == -1.0
# Count the number of masked elements
num_masked_elements = mask.sum().item()
print("Number of masked elements:", num_masked_elements)
print("Total number of elements in idf_tensor:", idf_tensor.shape[0])

# Normalize idf_tensor to match the distribution of the original U[:, 0]
original_mean, original_std = U[:, 0].mean(), U[:, 0].std()
idf_mean, idf_std = idf_tensor.mean(), idf_tensor.std()
normalized_idf_tensor = (idf_tensor - idf_mean) / idf_std * original_std + original_mean

# Use the mask to retain original U[:, 0] values for entries where idf_tensor == -1.0
normalized_idf_tensor[mask] = U[:, 0][mask]

# Set the first column of U to the modified IDF values
U[:, 0] = normalized_idf_tensor

# Step 4: Reconstruct W_E with the modified U
W_E_modified = torch.mm(U, torch.diag(S)) @ Vh  # Reconstruct W_E with modified U
model.get_input_embeddings().weight.data = W_E_modified.to(model.device)  # Assign modified W_E back to model

# Step 5: Save the modified model and tokenizer if desired
modified_model_path = "modified_cross_encoder_model"
os.makedirs(modified_model_path, exist_ok=True)
model.save_pretrained(modified_model_path)

# Then start training

# Freeze all layers except the embeddings
for name, param in model.named_parameters():
    if 'embeddings' not in name:  # Freeze all layers except embeddings
        param.requires_grad = False

# Verify the number of trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Trainable Parameters: {trainable_params}')

# Training configuration
epochs = 10
learning_rate = 2e-5  # Adjust as necessary for training embeddings
optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)  # Optimizer only for trainable params
num_training_steps = epochs * len(dataloader_train)
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

# Track the best validation loss
best_val_loss = float("inf")
best_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/best_model_aligned_embedding_NDCG_fixedrandom"  # Path to save the best model
# Track the best NDCG score
best_ndcg = float("-inf")
fine_tuned_model_path = f"finetune_{DATASET}/neg_samples_{NUM_NEG_SAMPLES}_seed{SEED}/temp_model_epoch_NDCG_aligned_embedding_fixedrandom"

os.makedirs(best_model_path, exist_ok=True)
# Initial validation
evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')

# Training loop
for epoch in range(epochs):
    model.train()
    train_losses = []
    print(f"Epoch {epoch + 1}/{epochs}")

    for batch in tqdm(dataloader_train, desc="Train"):
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Zero gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(**batch)

        # Apply sigmoid to logits for binary relevance scores
        predictions = torch.sigmoid(outputs.logits).squeeze()

        # Calculate MSE loss with sigmoid-transformed predictions
        loss = nn.functional.mse_loss(predictions, batch['labels'].float())
        train_losses.append(loss.cpu().item())

        # Backward pass
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Print average training loss for the epoch
    avg_train_loss = mean(train_losses)
    print(f"Epoch: {epoch + 1} Training loss: {avg_train_loss:.2f}")

    # Save the model at the end of each epoch
    model.save_pretrained(fine_tuned_model_path)
    tokenizer.save_pretrained(fine_tuned_model_path)

    # Evaluate NDCG using the saved model path
    ndcg = evaluate_beir(fine_tuned_model_path, NUM_NEG_SAMPLES=NUM_NEG_SAMPLES)
    print(f"NDCG@1 for epoch {epoch + 1}: {ndcg:.4f}")
    evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')


    # Check if this NDCG score is the highest seen so far
    if ndcg > best_ndcg:
        best_ndcg = ndcg
        # Save the model with the highest NDCG
        model.save_pretrained(best_model_path)
        tokenizer.save_pretrained(best_model_path)
        print(f"New best model saved with NDCG@1: {ndcg:.4f}")

print("Training complete. Best model saved at:", best_model_path)
"""

'\nimport torch\nfrom torch import nn, optim\nfrom tqdm.auto import tqdm\nfrom transformers import AutoModelForSequenceClassification, get_linear_schedule_with_warmup\nfrom statistics import mean\nimport os\ntorch.backends.cudnn.deterministic = True\ntorch.backends.cudnn.benchmark = False\nseed = SEED\nrandom.seed(seed)\nnp.random.seed(seed)\ntorch.manual_seed(seed)\ntorch.cuda.manual_seed_all(seed)  # if using CUDA\n# Set up device\ndevice = torch.device(\'cuda\' if torch.cuda.is_available() else \'cpu\')\n\nmodel = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)\n\n# Initialize Embedding Weights with SVD-based IDF Modification\n# Step 1: Extract the original embedding matrix W_E\nW_E = model.get_input_embeddings().weight.detach().cpu()  # Original embedding matrix W_E\n\n# Step 2: Perform SVD on the embedding matrix W_E\nU, S, Vh = torch.linalg.svd(W_E, full_matrices=False)  # Perform SVD on W_E\n\n# Step 3: Load IDF scores and normalize them\nidf_file_path 